In [1]:
!python -V

Python 3.9.12


In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
!ls data/

green_tripdata_2022-01.parquet	yellow_tripdata_2022-01.parquet
green_tripdata_2022-02.parquet	yellow_tripdata_2022-02.parquet


In [7]:
df = pd.read_parquet('data/yellow_tripdata_2022-01.parquet')

##### Data Shape. Number of columns = 19

In [8]:
df.shape

(2463931, 19)

In [44]:
df = pd.read_parquet('data/yellow_tripdata_2022-01.parquet')

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

##### STD of duration

In [45]:
df['duration'].std()

46.44530513776499

##### Share of data after filtering

In [47]:
df[(df.duration >= 1) & (df.duration <= 60)].shape[0]/df.shape[0]

0.9827547930522406

In [48]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

##### LR rmse on train data

In [51]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.98619013597892

#### Shape after onehot

In [53]:
X_train.shape

(2421440, 515)

##### Reading train and test again

In [9]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [10]:
df_train = read_dataframe('data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('data/yellow_tripdata_2022-02.parquet')

In [14]:
len(df_train), len(df_val)

(2421440, 2918187)

In [13]:
df_train = df_train[['tpep_pickup_datetime', 'tpep_dropoff_datetime','PULocationID','DOLocationID','duration']]
df_val = df_val[['tpep_pickup_datetime', 'tpep_dropoff_datetime','PULocationID','DOLocationID','duration']]

In [15]:
categorical = ['PULocationID', 'DOLocationID']

In [16]:
# categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [17]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

##### RMSE for train and val data

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
print(mean_squared_error(y_train, y_pred, squared=False))

y_pred = lr.predict(X_val)

print(mean_squared_error(y_val, y_pred, squared=False))

6.98619013597892
7.786389459975042
